In [1]:
from itertools import combinations
from z3 import *
import numpy as np

In [2]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one(bool_vars):
    return at_most_one(bool_vars) + [at_least_one(bool_vars)]

In [3]:
## Instance 1
m = 2
n = 6
l = [ 15,10 ]
s = [ 3,2,6,5,4,4 ]
D = [[0,3,4,5,6,6,2], [3,0,1,4,5,7,3], [4,1,0,5,6,6,4], [4,4,5,0,3,3,2], [6,7,8,3,0,2,4], [6,7,8,3,2,0,4], [2,3,4,3,4,4,0]]

instance = {
    'm' : m,
    'n' : n,
    'l' : l,
    's' : s,
    'D' : D
}

In [98]:
## Instance 2
m = 6
n = 9
l = [ 190,185,185,190,195,185 ]
s = [ 11,11,23,16,2,1,24,14,20 ]
D = [[0,199,119,28,179,77,145,61,123,87],
     [199,0,81,206,38,122,55,138,76,113],
     [119,81,0,126,69,121,26,117,91,32],
     [28,206,126,0,186,84,152,68,130,94],
     [169,38,79,176,0,92,58,108,46,98],
     [77,122,121,84,102,0,100,16,46,96],
     [145,55,26,152,58,100,0,91,70,58],
     [61,138,113,68,118,16,91,0,62,87],
     [123,76,91,130,56,46,70,62,0,66],
     [87,113,32,94,94,96,58,87,66,0]
     ]

instance = {
    'm' : m,
    'n' : n,
    'l' : l,
    's' : s,
    'D' : D
}

In [4]:
def solve_assignment_sat(m, n, loads, sizes, distances):
    PACKS = range(n)
    COURIERS = range(m)
    DEPOT = (n)
    LOCATIONS = range(DEPOT+1)

    assignments = [[Bool(f"a_{p}_{c}") for c in COURIERS] for p in PACKS]


    solver_assignments = Solver()

    ## Capacity constraint - assure that sum of weights of a singular courier is under its load limit
    for c in COURIERS:
        sum_load = Sum([If(assignments[p][c], sizes[p], 0) for p in PACKS])
        # sum_load = 0
        # for p in range(n):
        #     if assignments[p, c]==True:
        #         sum_load += sizes[p]
        solver_assignments.add(sum_load <= loads[c])
    
    ## Capacity constraint - ensure that each pack is delivered only by a courier
    for p in PACKS:
        solver_assignments.add(exactly_one([assignments[p][c] for c in COURIERS]))
    
    

    # Solve the problem
    # solver_assignments.set("timeout", 10)
    if solver_assignments.check() == sat:
        model = solver_assignments.model()

        # ass_list = []
        # for p in PACKS:
        #     for c in COURIERS:
        #         if model.evaluate(assignments[p][c]):
        #             ass_list.append(c+1)
        return [[model.evaluate(assignments[p][c]) for c in COURIERS] for p in PACKS]
    return None

In [14]:
def solve_path_sat(m, n, loads, sizes, distances, assignments_result, objective_old):
    PACKS = range(n)
    COURIERS = range(m)
    DEPOT = (n)
    LOCATIONS = range(DEPOT+1)

    assignments = [[Bool(f"a_{p}_{c}") for c in COURIERS] for p in PACKS]
    paths = [[[Bool(f'p_{c}_{loc1}_{loc2}') for loc2 in LOCATIONS] for loc1 in LOCATIONS] for c in COURIERS]


    solver_paths = Solver()

    ## Set assignments obtained through precedent model
    for p in PACKS:
        for c in COURIERS:
            solver_paths.add(assignments_result[p][c] == assignments[p][c])

    ## Path related constraints
    # 1 - Nel path deve esistere uno step con valore del pacco True con indice = DEPOT (n+1)
    for c in COURIERS:
        solver_paths.add(Implies(at_least_one([assignments[p][c] for p in PACKS]), paths[c][DEPOT][DEPOT]==False))
        solver_paths.add(Implies(Not(at_least_one([assignments[p][c] for p in PACKS])), paths[c][DEPOT][DEPOT]==True))

        for p in PACKS:
            solver_paths.add(Implies(assignments[p][c], paths[c][p][p]==False))
            solver_paths.add(Implies(Not(assignments[p][c]), paths[c][p][p]==True))

    # 2 - ensuring pack is delivered by a single courier only once
    for c in COURIERS:
        for loc in LOCATIONS:
            solver_paths.add(exactly_one([paths[c][loc][ind] for ind in LOCATIONS]))

    ## Subcircuit constraint
    # 1 - alldifferent per ogni path di ogni courier
    for c in COURIERS:
        for loc in LOCATIONS:
            solver_paths.add(exactly_one([paths[c][ind][loc] for ind in LOCATIONS]))

    # 2 - subtour elimination
    # Variables: u[i] to prevent subtours
    u = [[[Bool(f"u_{i}_{j}_{k}") for k in PACKS] for j in PACKS] for i in COURIERS]

    # Constraining assignment of truth value of u decision variable
    for c in COURIERS:
        # u del primo pacco = 1
        for p in PACKS:
            solver_paths.add(Implies(paths[c][DEPOT][p], u[c][p][0]==True))

        # u_j >= u_i + 1
        for p1 in PACKS:
            for p2 in PACKS:
                if p1 == p2:
                    continue

                for k in PACKS:
                    solver_paths.add(Implies(And(paths[c][p1][p2], u[c][p1][k]), And(exactly_one([u[c][p2][i] for i in range(k+1, n)]))))

    ## Exatcly one true value for each u_p1
    for c in COURIERS:
        for p1 in PACKS:
            solver_paths.add(exactly_one(u[c][p1]))

    # Applying MTZ formulation constraint
    # u_i - u_j + 1 <= (n - 1) * (1 - paths[c, i, j])
    for c in COURIERS:
        for p1 in PACKS: # i
            for p2 in PACKS: #j
                if p1 == p2:
                    continue

                for k1 in PACKS:
                    for k2 in PACKS:
                        solver_paths.add(Implies(And(u[c][p1][k1], u[c][p2][k2]), k1 - k2 + 1 <= (n-1) * (1-If(paths[c][p1][p2], 1, 0))))


    while solver_paths.check() == sat:
        model = solver_paths.model()


        objective = max([
            sum([distances[loc1][loc2] if model.evaluate(paths[c][loc1][loc2]) else 0
                for loc1 in LOCATIONS for loc2 in LOCATIONS if loc1!=loc2])
            for c in COURIERS
            ])
        
        solver_paths.add(objective < objective_old)

        print(f'Objective: {objective}')

        for c in COURIERS:
            total_distance = Sum([If(paths[c][loc1][loc2], distances[loc1][loc2], 0) for loc1 in LOCATIONS for loc2 in LOCATIONS if loc1!=loc2])

            solver_paths.add(total_distance < objective)

    return objective



In [17]:

set_option("sat.random_seed", 42)

final_result = 100000000
i = 0
while True:

    # Obtain a first assignment to variable assignment (=> which courier deliver each pack)
    print(f'({i+1}) - try assignment')
    assignment_result = solve_assignment_sat(instance['m'], instance['n'], instance['l'], instance['s'], instance['D'])

    # If this first problem is unsat => it doesn't find a solution: break
    if assignment_result == None:
        print(f'({i+1}) - assignment break')
        break
    
    # Initialize final result old with last value
    final_result_old=final_result

    # Obtain actual final result (Objective)
    print(f'({i+1}) - try paths')
    final_result = solve_path_sat(instance['m'], instance['n'], instance['l'], instance['s'], instance['D'], assignment_result, final_result_old)

    # if new final result worse than before: break
    if final_result > final_result_old:
        print(f'({i+1}) - paths break')
        break

    i += 1



(1) - try assignment
(1) - try paths
Objective: 16
Objective: 15
(2) - try assignment
(2) - try paths
Objective: 18
(2) - paths break


In [16]:
final_result

16